<a href="https://colab.research.google.com/github/ljrn8/NIDS-HPO/blob/main/colab%20notebooks/initial_model_testing_may16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
from pathlib import Path

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv(root / 'ids_datasets' / 'interm' / 'full_cicids_cleaned.csv')

In [ ]:
df.info()

## Preprocessing

In [ ]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

le = LabelEncoder()
df['Label'] = le.fit_transform(df['Label'])

X, y = df.drop('Label', axis=1), df['Label']

ss = StandardScaler()
X = ss.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=0)

In [ ]:
X.shape, np.unique(y)

((2827876, 71),
 array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14]))

## Training

In [ ]:
from tensorflow import keras
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dense, Input, Flatten

def get_model(input_shape, classes):
  model = keras.Sequential([
    Input(shape=input_shape),
    Conv1D(64, kernel_size=6, activation='relu'),
    MaxPooling1D(),
    Conv1D(64, kernel_size=6, activation='relu'),
    Flatten(),
    Dense(32, activation='relu'),
    Dense(classes, activation='softmax'),
  ])

  model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(), # expect label encoding
              metrics=['accuracy'])

  return model

model = get_model(input_shape=(X.shape[1], 1), classes=len(np.unique(y)))
model.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_12 (Conv1D)              │ (None, 66, 64)         │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_6 (MaxPooling1D)  │ (None, 33, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_13 (Conv1D)              │ (None, 28, 64)         │        24,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_6 (Flatten)             │ (None, 1792)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 32)             │        57,376 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 15)             │           495 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 82,959 (324.06 KB)

 Trainable params: 82,959 (324.06 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_test, y_test))

Epoch 1/10
70697/70697 ━━━━━━━━━━━━━━━━━━━━ 808s 11ms/step - accuracy: 0.9959 - loss: 0.0190 - val_accuracy: 0.9994 - val_loss: 0.0023
Epoch 2/10
70697/70697 ━━━━━━━━━━━━━━━━━━━━ 817s 11ms/step - accuracy: 0.9994 - loss: 0.0029 - val_accuracy: 0.9993 - val_loss: 0.0024
Epoch 3/10
48041/70697 ━━━━━━━━━━━━━━━━━━━━ 3:43 10ms/step - accuracy: 0.9994 - loss: 0.0026

KeyboardInterrupt: 

## Evaluation

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history)
plt.show()

In [ ]:
# why CNN -> Try deep belief network instead ??
# add this notebook to repo